In [ ]:
import numpy as np
from matplotlib.pyplot import *
from scipy import *
import scipy
from scipy.io import mmread
import pylab
import time
import scipy.sparse.linalg as spl
import matplotlib.pyplot as plt

In [ ]:
def cg(A,b,x0,maxit):
    print("my cg!")
    x = x0
    r = b-A@x
    p = r
    E=[]
    for i in range(maxit):
        r_dot_r = r.T@r
        alpha = r_dot_r/(p.T @ A @ p)
        x+= alpha*p
        if (i+1)%1==0:
            r = b-A@x
        else:
            r     -= alpha*(A@p)
        beta= (r.T@r)/r_dot_r
        p       = r + beta*p
        E.append(r_dot_r)
        #p       = r + beta*p
    return x,E

def cg2(A,b,x0,maxit):
    print("my cg!")
    x = x0
    r = b-A@x
    p = r
    E=[]
    for i in range(maxit):
        r_dot_r = r.T@r
        alpha = r_dot_r/(p.T @ A @ p)
        x+= alpha*p
        if (i+1)%1==0:
            r = b-A@x
        else:
            r     -= alpha*(A@p)
        beta= (r.T@r)/r_dot_r
        p       = r + beta*p
        p       = r + beta*p
        E.append(r_dot_r)
    return x,E
    

In [ ]:
def computeBasis(A,x,s,alp,bet,gam):
    n = A.shape[0]
    V = np.zeros((n,s+1))
    V[:,0] = x
    if(s>0):
        V[:,1] = (1/gam[0])*(A @ V[:,0]-alp[0]*V[:,0]) 
        for j in range(2,s+1):
            V[:,j] = (1/gam[j-1])*( (A @ V[:,j-1]-alp[j-1]*V[:,j-1]) - bet[j-2]*V[:,j-2])
    return V

def generate_C(s,alp,bet,gam):
    A = np.diag(alp,0) + np.diag(bet,1) + np.diag(gam[:-1],-1)
    B = np.zeros((1,s))
    B[0,-1]= gam[-1]
    C = np.append(A,B,axis=0)
    return C

def cacg(A,b,s,x0,outer_it):
    N = A.shape[0]
    r0 = b - A @ x0;
    p0 = r0;
    alp = np.zeros(s)
    bet = np.zeros(s-1)
    gam = np.ones(s)
    C = generate_C(s,alp,bet,gam)
    x = x0
    p = p0
    r = r0
    E=[]
    for it in range(outer_it):
        P = computeBasis(A,p,s,alp,bet,gam)
        R = computeBasis(A,r,s-1,alp,bet,gam)
        t1= np.append(C,np.zeros((s+1,1)),axis=1)
        t2= np.append(C[:-1,:-1],np.zeros((s,1)),axis=1)
        B = scipy.linalg.block_diag(t1,t2)
        #
        p_c = np.zeros(2*s+1)
        r_c = np.zeros(2*s+1)
        x_c = np.zeros(2*s+1)
        #
        p_c[0]=1
        r_c[s+1]=1
        #
        V = np.append(P,R,axis=1)
        G = V.T @ V
        #
        r_dot_r = r_c.T @ G @ r_c
        for j in range(s):
            alpha = r_dot_r / (p_c.T @ G @ B @ p_c)
            x_c   = x_c + alpha* p_c
            r_c   = r_c - alpha* B @ p_c
            beta  = 1/r_dot_r
            r_dot_r=r_c.T @ G @ r_c
            beta  = beta*r_dot_r
            if abs(beta)<10e-16:
                return x + V @ x_c
            #
            p_c   = r_c + beta * p_c
        x = x + V @ x_c
        r = b-A@x
        E.append(r.T@r)
        p = r + beta*V@p_c
        #print("it,residue: ",it,r.T@r)
        #print("max entry of residue:",max(r))
        if abs(r.T@r)<10e-24:
            break;
    return x,E

In [ ]:
#test simple
A= np.array([[2,3],[3,6]])
b= np.array([3,-3])
true_x = np.linalg.inv(A)@b
s=2
x0=np.zeros(2)
print(true_x)
cacg(A,b,s,x0,20)

In [ ]:
#test mid
dim = 1000
A=np.random.random((dim,dim))
A=A.T@A
true_x = np.random.rand(dim)
b= A@true_x

In [ ]:
s=2
outer_it= int(np.ceil(2000/s))
x0=np.zeros(dim)
ans = cacg(A,b,s,x0,outer_it)
print("E:",(b - A @ ans).T @ (b- A @ ans))

In [ ]:
# real test
dim=1138
A = mmread("./1138_bus.mtx")
A = A.tocsr()
true_x = np.random.rand(1138)
b = A @ true_x
x0=np.zeros(dim)

In [ ]:
#cacg
s=1
outer_it= int(np.ceil(1000/s))

x0=np.zeros(dim)
start = time.time()
ans = cacg(A,b,s,x0,outer_it)
end = time.time()
print(end - start)
print("E:",(b - A @ ans).T @ (b- A @ ans))
print(max((b - A @ ans)))
print(max(abs(true_x-ans)))

In [ ]:
#scipy benchmark
start = time.time()
sci_x = spl.cg(A,b,maxiter=1000)
end = time.time()
print(end - start)
print(sci_x[1])
print("E:",(b - A @ sci_x[0]).T @ (b- A @ sci_x[0]))
print(max((b - A @ sci_x[0])))
print(max(abs(true_x-sci_x[0])))

In [ ]:
# cg 
start = time.time()
x = cg(A,b,1000)
end = time.time()
print(end - start)
print("E:",(b - A @ x).T @ (b- A @ x))
print(x)
print(sci_x[0])
print(true_x)

In [ ]:
# test cant
dim= 62451
A = mmread("/Users/user/test/cant.mtx")
A = A.tocsr()
true_x = np.random.rand(dim)
b = A @ true_x
x0=np.zeros(dim)
# To Do...

In [ ]:
#scipy benchmark
start = time.time()
sci_x = spl.cg(A,b,tol=10e-20,maxiter=2000)
end = time.time()
print(end - start)
print(sci_x[1])
print("E:",(b - A @ sci_x[0]).T @ (b- A @ sci_x[0]))

In [ ]:
A = mmread("./cant.mtx")
A = A.tocsr()
n = A.shape[0]

In [ ]:
sol = np.zeros((n,1))
for i in range(n):
    sol[i]=np.random.randint(-50,50)/100

In [ ]:
b = A @ sol

In [ ]:
x = np.zeros((n,1))
r = b.copy()
p = r.copy()
x = np.zeros((n,1))
r_dot_r = r.T@r
for i in range(2000):
    r_dot_r = r.T@r
    alpha   = r_dot_r/(p.T @ A @ p)
    x      += alpha*p
    if (i+1)%1==0:
        r = b-A@x
    else:
        r       -= alpha*(A@p)
    beta    = (r.T@r)/r_dot_r
    p       = r + beta*p
    print(r_dot_r)
#    E.append(max(A@x-b))

In [ ]:
#scipy benchmark
start = time.time()
sci_x = spl.cg(A,b,tol=10e-30,maxiter=2000)
print(sci_x[1])
end = time.time()
print(end - start)

In [ ]:
#cacg
s=1
outer_it= int(np.ceil(2000/s))

x0=np.zeros(dim)
start = time.time()
ans,EE2 = cacg(A,b,s,x0,outer_it)
end = time.time()
print(end - start)
print("E:",(b - A @ ans).T @ (b- A @ ans))
print(max((b - A @ ans)))
print(max(abs(true_x-ans)))

In [ ]:
fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)
x=[]
for i in range(2000):x.append(i)
x=np.array(x)
#ax.plot(x,EE2,c='b',label='original CG',fillstyle='none')

ax.plot(5*x,E_k1,c='b',label='original CG',fillstyle='none')
ax.set_yscale('log')
plt.xlabel('number of iterations')
plt.ylabel('residue squared')
plt.title('CA-CG smoothing with ')

plt.grid()
plt.legend(loc=1)
plt.show()


In [ ]:
E_k1 = EE

In [ ]:
#cacg
s=5
outer_it= int(np.ceil(10000/s))

x0=np.zeros(dim)
start = time.time()
ans,EE = cacg(A,b,s,x0,outer_it)
end = time.time()
print(end - start)
print("E:",(b - A @ ans).T @ (b- A @ ans))
print(max((b - A @ ans)))
print(max(abs(true_x-ans)))

In [ ]:
#cacg
s=1
outer_it= int(np.ceil(10000/s))

x0=np.zeros(dim)
start = time.time()
ans,EE2 = cacg(A,b,s,x0,outer_it)
end = time.time()
print("time is ",end - start)
print("E:",(b - A @ ans).T @ (b- A @ ans))
print(max((b - A @ ans)))
print(max(abs(true_x-ans)))

In [ ]:
fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)
x=[]
for i in range(10000):x.append(i)
x=np.array(x)
ax.plot(x,EE2,c='b',label='original CG',fillstyle='none')
ax.set_yscale('log')
plt.xlabel('number of iterations')
plt.ylabel('residue squared')
plt.title('CA-CG smoothing with ')

plt.grid()
plt.legend(loc=1)
plt.show()

In [ ]:
fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)
x=[]
y=[]
for i in range(10000):x.append(i)
for i in range(2000):y.append(i)
x=np.array(x)
y=np.array(y)
ax.plot(x,EE2,c='b',label='CA-CG with s = 1',fillstyle='none')
ax.plot(5*y,E_k1,c='g',label='CA-CG with s = 5',fillstyle='none')
ax.set_yscale('log')
plt.xlabel('number of iterations')
plt.ylabel('residue squared')
plt.title('smoothing CA-CG convergence')

plt.grid()
plt.legend(loc=1)
plt.show()

In [ ]:
#cacg
s=1
outer_it= int(np.ceil(10000/s))
x0=np.zeros(dim)
start = time.time()
_,_ = cacg(A,b,s,x0,outer_it)
end = time.time()
print("time is ",end - start)

In [ ]:
#cacg
s=2
outer_it= int(np.ceil(10000/s))
x0=np.zeros(dim)
start = time.time()
_,_ = cacg(A,b,s,x0,outer_it)
end = time.time()
print("time is ",end - start)

In [ ]:
#cacg
s=4
outer_it= int(np.ceil(10000/s))
x0=np.zeros(dim)
start = time.time()
_,_ = cacg(A,b,s,x0,outer_it)
end = time.time()
print("time is ",end - start)

In [ ]:
#scipy benchmark
start = time.time()
sci_x = spl.cg(A,b,tol=10e-30,maxiter=10000)
print(sci_x[1])
end = time.time()
print(end - start)